<center><h2>Exercices pratiques de webscraping</h2></center>
<br/>

In [1]:
# Installation des librairies tierces utilisées.
import pip
for pkg in ['requests', 'bs4']:
    pip.main(['install', pkg])

### Exercice 1 - Suivi de la "Une" du site Le Monde

L'objectif est ici de récupérer la liste des articles en "Une" du site http://www.lemonde.fr/ de manière automatisée.

_Note : le fichier `robots.txt` du site Le Monde interdit la navigation automatisée, aussi veillera-t-on à limiter
 au maximum le nombre de requêtes effectuées, dans cet exercice comme dans les suivants. Pour ce faire, mieux vaut
 séparer les cellules vouées au téléchargement des pages à analyser de celles où l'extraction d'information à lieu,
 puisque cette dernière est par nature soumise à un processus d'essais/erreurs, sans qu'il ne soit nécessaire de
 télécharger la page à chaque itération._


####  1.0 Récupérer la page d'accueil du site

In [2]:
import requests

response = requests.get('http://www.lemonde.fr/')
if response.status_code != 200:
    print('Échec de la requête: statut %s.' % response.status_code)

#### 1.1 Récupérer l'article en "Une".

Le premier objectif est de récupérer l'article en "Une" du journal _Le Monde_. Il s'agit d'extraire son titre, la ou les phrases d'accroche associées et l'url de la page de l'article.

**Correction** :

* L'étude de la page permet de repérer que l'article en Une se trouve dans une balise `<article>` de classe spécifique `titre_une`. Ces deux critères permettent alors de récupérer cet article à l'aide de `BeautifulSoup`.

* Au sein de la balise extraite se trouvent des sous-balises structurant les informations ciblées : le titre est dans une balise `<h1>`, le texte dans une balise `<p>` et l'url (relative) dans la propriété `href` d'une balise `<a>`.

* Optionnellement, on peut nettoyer le texte extrait.

In [3]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, 'html.parser')
article_une = soup.find('article', {'class': 'titre_une'})

In [4]:
import re

def clean_text(string):
    """Fonction de nettoyage textuel sommaire."""
    # Remplace les tabluations, sauts de lignes, etc. par des espaces.
    string = re.sub('\n|\r|\t|\xa0', ' ', string)
    # Retire les ' .' (séparateurs utilisés dans `soup.get_text`) inappropriés.
    string = re.sub('^\.', '', string.replace(' .', ''))
    # Retire les espaces inappropriés.
    return re.sub('  +', ' ', string).strip(' ')
    

In [5]:
{
    'titre': clean_text(article_une.find('h1').text),
    'texte': clean_text(article_une.find('p').text),
    'url': 'https://www.lemonde.fr' + article_une.find('a')['href']
}

{'texte': 'Les Etats membres de l’UE, réunis en comité d’appel à Bruxelles, ont voté lundi à une majorité qualifiée de 18 Etats.',
 'titre': 'En direct : l’Union européenne renouvelle pour cinq ans la licence d’exploitation du glyphosate Live',
 'url': 'https://www.lemonde.fr/planete/live/2017/11/27/en-direct-l-union-europeenne-statue-sur-la-prolongation-du-glyphosate_5221023_3244.html'}

#### 1.2 Récupérer les articles en tête de section

Outre la Une, certains articles sont mis en avant sur la page d'accueil du journal _Le Monde_ par l'inclusion d'un petit texte d'accroche. Le second objectif consiste à repérer ces articles et à extraire les mêmes informations que pour la Une (titre, url et descriptif).

Attention, certaines urls ne sont pas faciles à identifier, du fait d'une technologie particulière permettant leur chargement lors du rendu de la page. Dans un premier temps, chercher à ne récupérer les urls que lorsqu'elles sont "faciles" à identifier. Optionnellement, chercher une astuce pour les retrouver (une proposition se trouve dans le corrigé de ce notebook).

**Correction** :

* Les articles ciblés font, comme la Une, l'objet d'une structuration au sein d'une balise `<article>` de classe spécifique `img_tt_chapo`.


* Les informations que l'on souhaite extraire sont structurées selon des sous-balises similaires au cas de la _Une_, si ce n'est que le titre est dans une balise `<h2>` et non `<h1>`.


* Dans certains cas cependant, l'url n'est pas directement accessible, car elle est chargée dynamiquement par un script à partir de l'attribut `data-href` d'une balise `<span>`. L'astuce proposée est alors de chercher parmi l'ensemble des urls disponibles sur la page s'il en est une qui contient les premiers mots du titre de l'article. Cette solution s'appuie sur la structure particulière des urls des articles sur le site du _Monde_, et n'est ni généralisable en toutes circonstances, ni une solution optimale ; sans doute peut-on faire mieux.

In [6]:
# Version "simple" de la solution, avec urls manquantes.
[
    {
        'titre': clean_text(article.find('h2').text),
        'texte': clean_text(article.find('p').text),
        'url': (
            ('https://www.lemonde.fr' + article.find('a')['href'])
            if article.find('a') is not None else ''
        )
    }
    for article in soup.find_all('article', {'class': 'img_tt_chapo'})
]

[{'texte': 'L’archevêque de Rangoun a recommandé au pape François, en visite en Birmanie et au Bangladesh, de ne pas utiliser le terme « Rohingya » pour évoquer la minorité musulmane persécutée.',
  'titre': 'Pourquoi le terme de « Rohingya » est tabou en Birmanie',
  'url': 'https://www.lemonde.fr/asie-pacifique/article/2017/11/27/pourquoi-le-terme-de-rohingya-est-tabou-en-birmanie_5221006_3216.html'},
 {'texte': 'Editorial. L’appel de Macron à faire de l’égalité hommes-femmes une « grande cause du quinquennat » est assorti de mesures, mais pas de nouveaux moyens.',
  'titre': 'L’antisexisme, grande cause petits moyens 18',
  'url': 'https://www.lemonde.fr/idees/article/2017/11/27/la-grande-cause-de-l-antisexisme_5220938_3232.html'},
 {'texte': 'Notre journaliste Jean-Philippe Rémy a répondu aux internautes sur la démission forcée du président Robert Mugabe après trente-sept ans à la tête du pays.',
  'titre': '« L’enjeu pour le nouveau pouvoir, c’est de changer les règles pour faire 

In [7]:
# Fonctions auxiliaires pour trouver certaines des urls manquantes.

def get_href_from_title(titre, soup):
    """Cherche un lien parmi ceux d'une page concordant avec un titre donné."""
    # Nettoyage du titre.
    titre = retirer_accents(titre.lower())
    titre = re.sub('^ | $', '', re.sub('  +', ' ', re.sub('[^a-z ]', '', titre)))
    # Sous-sélection et formattage des trois premiers termes du titre.
    title_part = '-'.join(titre.split(' ')[:3])
    # Itération sur les urls liés dans la page, jusqu'à en trouver une qui concorde.
    for anchor in soup.find_all('a'):
        if title_part in anchor.get('href', ''):
            return 'https://www.lemonde.fr' + anchor['href']

        
def retirer_accents(text):
    """Remplace les caractères accentués par leur équivalent sans accent."""
    charmap = [
            ('àâ', 'a'), ('ç', 'c'), ('éèêë', 'e'),
            ('îï', 'i'), ('ôö', 'o'), ('ùûü', 'u')
        ]
    for character in charmap:
        text = re.sub('[%s]' % character[0], character[1], text)
    return text

In [8]:
# Version "complexe" utilisant les fonctions précédentes.

def parse_head_article(article, soup):
    """Extrait le titre, le texte et l'url associés à un article "mis en avant"."""
    parsed = {
        'titre': clean_text(article.find('h2').text),
        'texte': clean_text(article.find('p').text)
    }
    parsed['url'] = (
        get_href_from_title(parsed['titre'], soup)
        if article.find('a') is None
        else 'https://www.lemonde.fr' + article.find('a')['href']
    )
    return parsed

articles = [
    parse_head_article(article, soup)
    for article in soup.find_all('article', {'class': 'img_tt_chapo'})
]

In [9]:
# Affichage "lisible" des résultats.

def format_article(article_dict):
    return '%s\n\n%s\n\n%s' % (article_dict['titre'], article_dict['texte'], article_dict['url'])


print('\n\n\n'.join(map(format_article, articles)))

Pourquoi le terme de « Rohingya » est tabou en Birmanie

L’archevêque de Rangoun a recommandé au pape François, en visite en Birmanie et au Bangladesh, de ne pas utiliser le terme « Rohingya » pour évoquer la minorité musulmane persécutée.

https://www.lemonde.fr/asie-pacifique/article/2017/11/27/pourquoi-le-terme-de-rohingya-est-tabou-en-birmanie_5221006_3216.html


L’antisexisme, grande cause petits moyens 18

Editorial. L’appel de Macron à faire de l’égalité hommes-femmes une « grande cause du quinquennat » est assorti de mesures, mais pas de nouveaux moyens.

https://www.lemonde.fr/idees/article/2017/11/27/la-grande-cause-de-l-antisexisme_5220938_3232.html


« L’enjeu pour le nouveau pouvoir, c’est de changer les règles pour faire décoller le Zimbabwe »

Notre journaliste Jean-Philippe Rémy a répondu aux internautes sur la démission forcée du président Robert Mugabe après trente-sept ans à la tête du pays.

https://www.lemonde.fr/afrique/article/2017/11/27/l-enjeu-pour-le-nouveau-p

#### 1.3 Récupérer les titres et adresses de tous les articles listés sur la page d'accueil

La page d'accueil du journal _Le Monde_ comporte des liens vers de nombreux articles, différemment mis en avant mais se composant _a minima_ d'un titre et d'un lien pointant vers l'url de la page de l'article. Le dernier objectif consiste à repérer ces articles dans le html récupéré, et à extraire pour chacun son titre et son url.

**Correction** :

* Tous les articles sont rangés dans la section "articles" du site, une solution simple est donc d'extraire l'ensemble des urls présents dans le html (attributs `href` des balises `<a>`), et de filtrer celles répondant à ce schéma.


In [10]:
def parse_lien(tag):
    """Parse un lien d'article sur le site du Monde."""
    url = tag['href']
    if tag.text != '':
        text = tag.text
    elif tag.find('h1') is not None:
        text = tag.find('h1').text
    elif tag.find('h2') is not None:
        text = tag.find('h2').text
    else:
        text = ''
    return {re.sub('^\d\d:\d\d', '', clean_text(text)): url}

In [11]:
liens = [parse_lien(a) for a in soup.find_all('a') if '/article/' in a['href']]

print('\n\n'.join(map(str, liens)))

{'La Commission européenne autorise l’utilisation du glyphosate jusqu’en 2022': '/europe/article/2017/11/27/la-commission-europeenne-autorise-l-utilisation-du-glyphosate-jusqu-en-2022_5221037_3214.html'}

{'Quand Monsanto qualifiait de « science pourrie » la dernière étude dédouanant le glyphosate': '/planete/article/2017/11/26/quand-monsanto-qualifiait-de-science-pourrie-la-derniere-etude-dedouanant-le-glyphosate_5220720_3244.html'}

{'Après deux ans de négociations, l’UE doit statuer sur le glyphosate': '/planete/article/2017/11/27/glyphosate-l-europe-en-echec_5220871_3244.html'}

{'Glyphosate : révélations sur les failles de l’expertise européenne': '/planete/article/2017/11/26/glyphosate-revelations-sur-les-failles-de-l-expertise-europeenne_5220696_3244.html'}

{'Pourquoi le terme de « Rohingya » est tabou en Birmanie': '/asie-pacifique/article/2017/11/27/pourquoi-le-terme-de-rohingya-est-tabou-en-birmanie_5221006_3216.html'}

{'La question du respect des minorités au cœur du voyag

### Exercice 2 - Récupérer les résultats d'une recherche Google (ou Bing, ou DuckDuckGo...)

Si la légalité de la pratique est encore une fois à vérifier (le _robots.txt_ de Google l'interdit, par exemple - au profit d'un module spécifique limitant drastiquement le nombre de requêtes, sauf à payer pour celles-ci), il peut être intéressant d'effectuer une recherche _via_ un moteur de recherche et de récupérer automatiquement les urls listées. Ceci peut en particulier permettre de contourner l'absence de moteur de recherche sur un site (ou la difficulté à manipuler celui-ci sans recourir à un navigateur).

**Note** : Google est hégémonique (pour le meilleur et pour le pire), et est le premier mentionné dans la suite de l'exercice. Bing (lui aussi produit d'une grosse compagnie) et DuckDuckGo (qui veut du bien à votre vie privée) sont également traités dans le corrigé, et chacun devrait se sentir libre d'utiliser l'un de ces moteurs de recherche, ou tout autre service similaire de son choix.

#### 2.1 Effectuer une recherche Google (ou autre) avec Requests

La première étape (comme toujours) est de récupérer le html sur lequel on veut travailler - en l'occurence celui associé à une recherche donnée. En amont, il faut donc définir l'url correspondant à ladite requête. L'exercice consiste donc à rechercher le terme "insee" sur Google, Bing ou DuckDuckGo, et à récupérer le html associé à cette recherche.

_Conseil_ : dans le cas de Google, afin de "simplifier" l'url, utiliser https://encrypted.google.com/.

Certains paramètres (pour changer de page, régionaliser les résultats, etc.) ne sont pas toujours évidents à identifier, car ils peuvent être cachés. Lire les scripts présents dans le html peut aider, faire des tests dans un navigateur aussi. Le corrigé présente quelques options utiles identifiées pour les trois moteurs de recherche étudiés.

**Correction** :

Afin de trouver la structure de l'url à utiliser, le plus simple est d'effectuer une recherche quelconque sur le moteur de recherche, et de "déconstruire" celle-ci. Elle est toujours de la forme `<protocole>://<nom de domaine><éventuel raffinement>?<bloc d'options>`, où le dernier bloc est de la forme `<option>=<valeur>` avec un séparateur `&` entre chaque couple option / valeur.

* Google :
  * protocole, domaine et extension : https://encrypted.google.com/search
  * recherche : q=insee (pour une recherche sur le terme "insee")
  * page : 
  * régionalisation de la recherche : hl=fr (pour la France)
  * _exemple_ : https://encrypted.google.com/search?hl=fr&q=insee


* Bing :
  * protocole, domaine et extension : https://www.bing.com/search
  * recherche : q=insee (pour une recherche sur le terme "insee")
  * page: first=9 (pour obtenir dix résultats (dont publicités), à partir du neuvième (hors publicités))
  * _exemple_ : https://www.bing.com/search?q=insee


* DuckDuckGo : 
  * protocole, domaine et extension : https://duckduckgo.com/html
      * Note : dans le navigateur, la section html n'est pas utilisée. Elle est requêtée de manière sous-jacente de
        manière asynchrone _via_ un script JavaScript. En spécifiant l'adresse directement, on contourne cette
        spécificité, afin de récupérer "en clair" les résultats de la recherche.
  * recherche : q=insee
  * régionalisation de la recherche : kl=fr (pour la France)
  * page : s=30
    (pour obtenir trente (déterministe) résultats, à partir du trente-et-unième (spécifié) ; équivalent de la page 2)
  * spécifier un site auquel restreindre les résultats : site%3Awww.societe.com (pour une recherche sur www.societe.com),
    à inclure dans la recherche
  * _exemple_ : https://duckduckgo.com/html/?q=insee+site%3Awww.societe.com&kl=fr


**Note** : pour rechercher plusieurs termes à la fois (dans les trois cas précédents), séparer ces termes d'un `+` dans la valeur attribuée à l'option de recherche `q`.

In [12]:
import requests

response_google = requests.get("https://encrypted.google.com/search?hl=fr&q=insee")
if response_google.status_code != 200:
    print('Erreur avec Google: %s.' % response_google.status_code)

response_bing = requests.get("https://www.bing.com/search?q=insee")
if response_bing.status_code != 200:
    print('Erreur avec Bing: %s.' % response_bing.status_code)

response_duckduckgo = requests.get("https://duckduckgo.com/html/?q=insee&kl=fr")
if response_duckduckgo.status_code != 200:
    print('Erreur avec DuckDuckGo: %s.' % response_duckduckgo.status_code)

Erreur avec Google: 503.


#### 2.2 Extraire les résultats de la recherche

L'objectif est maintenant d'identifier les urls listées sur les pages de résultats précédemment requêtées. La solution diffère pour chaque moteur de recherche, et s'appuie sur les procédés de lecture de html et d'essais/erreurs déjà mobilisés.

**Correction** :

Dans chaque cas, les spécificités d'extraction des résultats diffèrent. Ci-dessous, une solution possible pour chacun des trois moteurs de recherche.

In [13]:
from bs4 import BeautifulSoup

soup_google = BeautifulSoup(response_google.text, 'html.parser')
soup_bing = BeautifulSoup(response_bing.text, 'html.parser')
soup_duckduckgo = BeautifulSoup(response_duckduckgo.text, 'html.parser')

In [14]:
liens_google = [
    a['href'][7:]
    for a in soup_google.find_all('a')
    if a.get('href', '').startswith('/url')
]

In [15]:
liens_bing = [
    li.find('a')['href']
    for li in soup_bing.find_all('li', {'class': 'b_algo'})
]

In [16]:
liens_duckduckgo = [
    a['href'][15:].replace('%3A', ':').replace('%2F', '/')
    for a in soup_duckduckgo.find_all('a', {'class': 'result__url'})
]

<center>**Voilà pour ces exercices introductifs ; bon courage pour la suite et la mise en pratique !**</center>